# Diabetic Retinopathy

In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
#from keras.applications.vgg16 import VGG16

#from keras.applications.xception import Xception
from keras.utils import Sequence
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
#from keras.applications.vgg16 import preprocess_input
from keras.applications.resnet50 import preprocess_input
import numpy as np
from keras.models import Model,Sequential
from keras.layers import Dense,Activation,Dropout,Flatten,AveragePooling2D
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import model_from_json
from PIL import Image

### Getting Training Labels

In [ ]:
data=pd.read_csv("../../datasets/Diabatic_Retinopathy/trainLabels.csv")

### Configuration for input shape

In [ ]:
size1=200
size2=300
channels=3
input_shape=(size1,size2,channels)
n_classes=5 

### Loading pretrained model

In [6]:

basemodel=ResNet50(weights='imagenet',include_top=False,input_shape=input_shape)
basemodel.summary()

94773248/94765736 [==============================] - 19s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 306, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 100, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 100, 150, 64) 256         conv1_conv[0][0]                 
_________________________

### Adding layers for fine tuning 

In [7]:

model=Sequential()
model.add(basemodel)
model.add(Flatten())
model.add(Dense(1000,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 10, 2048)       23587712  
_________________________________________________________________
flatten (Flatten)            (None, 143360)            0         
_________________________________________________________________
dense (Dense)                (None, 1000)              143361000 
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2002      
Total params: 166,950,714
Trainable params: 166,897,594
Non-trainable params: 53,120
_________________________________________________________________
None


### Fitting the model

In [ ]:

images=[]
label=[]
train=[]
mn1=5000
mn2=5000
count=0


a = np.arange(n_classes)
#print(a)

b = np.zeros((n_classes, n_classes))
b[np.arange(n_classes), a] = 1
print(b.shape)

#data=pd.read_csv("trainLabels.csv")
batch_size=1
count=0
k=False
t=0
flag=False
y=[]
def generator(batch_size):
        while(True):
            train=list(os.walk('../../datasets/Diabatic_Retinopathy/train'))[0][2]
            print(len(train))
        
            train2=list(map(lambda x:x.split('.')[0],train))
            
       
            for j in train2:
                try:
                
                    img=Image.open('../../datasets/Diabatic_Retinopathy/train/'+j+'.jpeg')
                    x=np.array(img.resize((size2,size1)).convert('RGB')).reshape(1,size1,size2,3)
                
                    ind=data[data['image']==j]["level"]
            
                    
                    global flag,count,t
                    if flag:
                        images=np.concatenate((images,x),axis=0)
                        label=np.concatenate((label,[b[ind]]),axis=0)
                    else:
                        flag=True
                        images=x
                    
                        label=np.array([b[ind]])
                    
                    count+=1
                    t+=1  
                    #print("count=",count," t=",t)
                    if(count==batch_size):
                        count=0
                        print(t)
                        flag=False
                        
                        yield images,label
                    
                     
                    
                        
                except Exception as e:
                    print(e)
                    continue
            

ep=(int)(35125/5)+1
#print(ep)

model.fit_generator(generator(5),epochs=5,steps_per_epoch=ep)


#.extend(model.predict_generator(generator(5),steps=ep))

### Saving model

In [ ]:
model_json = model.to_json()
with open("../../datasets/Diabatic_Retinopathy/resnet.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("../../datasets/Diabatic_Retinopathy/resnet.h5")


### Loading model

In [ ]:

 
load json and create model
json_file = open('../../datasets/Diabatic_Retinopathy/resnet.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

model.load_weights("../../datasets/Diabatic_Retinopathy/resnet.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
#model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

### Testing Model

In [ ]:
img=Image.open('../../datasets/Diabatic_Retinopathy/test/10207_left.jpeg')
x=np.array(img.resize((size2,size1)).convert('RGB')).reshape(1,size1,size2,3)

actual_label=data[data['image']=='10207_left']['level']

predicted_label=np.argmax(model.predict(x),axis=1)
